# RTML lab1 report - st121413

Your lab report should have the following sections:

1. Introduction: the background and goals of the lab
2. Methods: what you did, what parameters you tried, and so on
3. Results: what were the results
4. Conclusion: what did you learn from the lab, and what might be the next steps

In the results section, be sure to show training and validation loss as a function of training epochs. You'll also want to show results on a separate test set and give some analysis of the errors the classifier makes on the test set.

##  1. Introduction

The goal of this lab is to setup our machine to run SSH to puffer and connect to docker.

Then, we try to test wether our docker can run pytorch on GPUs.